In [1]:
import pandas
from bs4 import BeautifulSoup
import random
import time
from splinter import Browser

In [2]:
database="monsters.csv"
monsters = pandas.read_csv(database)
monsters = monsters.set_index('url')
monsters

,name,meta,str,dex,con,int,wis,cha,img,tidbits,descriptions
url,,,,,,,,,,,
https://www.dndbeyond.com/monsters/16895-giant-spider,Giant Spider,"Large Beast, Unaligned",14 (+2),16 (+3),12 (+1),2 (-4),11 (+0),4 (-3),https://www.dndbeyond.com/avatars/thumbnails/3...,"Skills: Stealth +7;Senses: Blindsight 10 ft., ...","<div class=""mon-stat-block__description-block-..."
https://www.dndbeyond.com/monsters/16841-dire-wolf,Dire Wolf,"Large Beast, Unaligned",17 (+3),15 (+2),15 (+2),3 (-4),12 (+1),7 (-2),https://www.dndbeyond.com/avatars/thumbnails/1...,"Skills: Perception +3, Stealth +4;Senses: Pass...","<div class=""mon-stat-block__description-block-..."


In [3]:
monsters_to_add = [
    'https://www.dndbeyond.com/monsters/16895-giant-spider', 
    'https://www.dndbeyond.com/monsters/16841-dire-wolf',
    'https://www.dndbeyond.com/monsters/16816-brown-bear',
    'https://www.dndbeyond.com/monsters/16991-rat',
    'https://www.dndbeyond.com/monsters/16891-giant-rat',
    'https://www.dndbeyond.com/monsters/16920-hawk',
    'https://www.dndbeyond.com/monsters/17049-warhorse',
    'https://www.dndbeyond.com/monsters/16997-riding-horse',
    'https://www.dndbeyond.com/monsters/16844-draft-horse',
    'https://www.dndbeyond.com/monsters/16874-giant-badger',
    'https://www.dndbeyond.com/monsters/16889-giant-owl',
    'https://www.dndbeyond.com/monsters/16900-giant-wolf-spider',
    'https://www.dndbeyond.com/monsters/16944-lion',
    'https://www.dndbeyond.com/monsters/17036-tiger',
    'https://www.dndbeyond.com/monsters/16976-panther',
    'https://www.dndbeyond.com/monsters/16881-giant-eagle',
    'https://www.dndbeyond.com/monsters/16974-owl'
]

In [4]:
def get_attribute(block):
    header = block.find("div", class_="ability-block__heading").text.strip()
    value = block.find("span", class_="ability-block__score").text.strip()
    mod = block.find("span", class_="ability-block__modifier")
    if mod != None:
        value += f" {mod.text.strip()}"
    return value


In [5]:
def get_monster(url):
    monster = {"url":url}
    found_monster=False
    main_block = None
    while not found_monster:
        browser_name = random.choice(['chrome', 'firefox'])
        browser = Browser(browser_name)
        time.sleep(random.random()*5)
        browser.visit(url)
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        main_block = soup.find("div", class_="mon-stat-block")
        found_monster = (main_block != None)
        time.sleep(random.random()*60)
        if not found_monster:
            time.sleep(random.random()*600+50)
        browser.quit()
    monster["name"] = main_block.find("div", class_="mon-stat-block__name").text.strip()
    monster["img"] = soup.find("img", class_="monster-image")["src"]
    monster["meta"] = main_block.find("div", class_="mon-stat-block__meta").text.strip()
    attributes_str = ""
    attributes = main_block.find("div", class_="mon-stat-block__attributes").find_all("div", class_="mon-stat-block__attribute")
    for attribute in attributes:
        label = attribute.find("span", class_="mon-stat-block__attribute-label").text.strip()
        value = attribute.find("span", class_="mon-stat-block__attribute-data-value").text.strip()
        dataextra = attribute.find("span", class_="mon-stat-block__attribute-data-extra")
        if not dataextra == None:
            value = f"{value} {dataextra.text.strip()}"
        attributes_str += f";{label}:{value}"
    monster["attributes"] = attributes_str[1:]
    tidbits_str = ""
    tidbits = main_block.find("div", class_="mon-stat-block__tidbits").find_all("div", class_="mon-stat-block__tidbit")
    for tidbit in tidbits:
        label = tidbit.find("span", class_="mon-stat-block__tidbit-label").text.strip()
        data = tidbit.find("span", class_="mon-stat-block__tidbit-data").text.strip()
        tidbits_str += f";{label}: {data}"
    monster["tidbits"] = tidbits_str[1:]
    stat_blocks = main_block.find("div", class_="mon-stat-block__stat-block")
    monster["str"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--str"))
    monster["dex"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--dex"))
    monster["con"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--con"))
    monster["int"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--int"))
    monster["wis"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--wis"))
    monster["cha"]=get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--cha"))
    descriptions_str = ""
    descriptions = main_block.find_all("div", class_="mon-stat-block__description-block")
    for description in descriptions:
        #heading = description.find("div", class_="mon-stat-block__description-block-heading").text.strip()
        content = description.find("div", class_="mon-stat-block__description-block-content")
        descriptions_str += ";;;" + str(content)
        monster["descriptions"] = descriptions_str[3:]
    return(monster)

In [6]:
def save_database(monsters, database):
    monsters["url"] = monsters.index
    monsters = monsters[['url','name', 'meta', "str","dex","con","int","wis","cha","img","tidbits","descriptions"]]
    monsters.to_csv(database, index=False)

In [7]:
def add_monster(monsters, url):
    if url in monsters.index:
        print(f"found {monsters.loc[url,'name']}, skipping")
        return(monsters)
    else:
        monster = get_monster(url)
        monster_df = pandas.DataFrame([monster]).set_index("url")
        return(pandas.concat([monsters, monster_df]))

In [ ]:
for monster_url in monsters_to_add:
    monsters = add_monster(monsters, monster_url)
    save_database(monsters, database)
monsters

found Giant Spider, skipping
found Dire Wolf, skipping


In [ ]:
save_database()